# Class 9: Corner Detection and Keypoint Matching

## Preliminaries

Run the cell below to download the course library and class resources.

In [ ]:
from urllib.request import urlretrieve

urlretrieve('https://drive.google.com/uc?export=download&id=1SiLnO91qJWKomBkGhKciJSG9Hcd_zRp7', 'sdx.zip')
!unzip -o 'sdx.zip'
!rm 'sdx.zip'

urlretrieve('https://drive.google.com/uc?export=download&id=19_wiL7cEmw_tEKROT9B-1NDxV9n7bhcP', '09.zip')
!unzip -o '09.zip'
!rm '09.zip'

Run the cell below to import the class modules.

If you get import warnings, try using **`Ctrl+M .`** to restart the kernel. *(notice there is a dot there)*

In [ ]:
import numpy as np
import cv2 as cv

from sdx import *

## Convenience functions

### Harris-Stephens corner detector

In [ ]:
def harris_stephens_points(image, limit=None):
    height, width = image.shape

    # Keep these parameters fixed. Thinking about
    # them will only give you analysis paralysis.
    harris = cv.cornerHarris(image, 3, 3, 0.05)

    # The cornerHarris function returns an array
    # with the response values for each pixel.
    # We will convert this to a list of points.

    points = []

    for y in range(height):
        for x in range(width):
            value = harris[y, x]

            # Negative values are edge responses
            # and zero values are flat responses.
            # We will only keep corner responses.
            if value > 0:
                points.append((y, x, value))

    # Sorting by the value: strong corners first.
    points.sort(key=lambda point: point[2], reverse=True)

    if limit is None:
        limit = len(points)
    return [(y, x) for y, x, value in points][:limit]

### Shi-Tomasi corner detector

In [ ]:
def shi_tomasi_points(image, limit=None):
    height, width = image.shape

    # Keep these parameters fixed. Thinking about
    # them will only give you analysis paralysis.
    gftt = cv.goodFeaturesToTrack(image, 0, 0.01, 0)

    # Unlike cornerHarris, the goodFeaturesToTrack
    # function returns a list of points already
    # sorted by corner strength. We just need to
    # clean up the output a bit for easier usage.
    points = []
    for point in gftt:
        x, y = point[0]
        points.append((round(y), round(x)))

    if limit is None:
        limit = len(points)
    return points[:limit]

### Draw keypoints over an image

In [ ]:
def draw_points(image, points):
    image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)

    for y, x in points:
        cv.circle(image, (x, y), 4, (0, 0, 255), 2)

    cv_imshow(image)

### Draw matches over two images

In [ ]:
def draw_matches(source, target, matches):
    s_height, s_width = source.shape
    t_height, t_width = target.shape

    height = max(s_height, t_height)
    shift = s_width + 1
    width = shift + t_width

    image = np.full((height, width), 255, np.float32)
    image[:s_height, :s_width] = source
    image[:t_height, (s_width + 1):width] = target

    image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)

    for s, t in matches:
        s_y, s_x = s
        t_y, t_x = t
        cv.line(image, (s_x, s_y), (shift + t_x, t_y), (0, 255, 0), 1)

    cv_imshow(image)

### Take a patch around a point

In [ ]:
def patch(image, y, x, size):
    radius = size // 2
    return image[(y - radius):(y + radius + 1), (x - radius):(x + radius + 1)]

## Functions you should change

### Calculate the distance between two patches

In [ ]:
from scipy.ndimage import rotate # Library to align the patch orientations

def calculate_distance(s_patch, t_patch, size):
    s_gradient_x, s_gradient_y = np.gradient(s_patch, axis=1), np.gradient(s_patch, axis=0)
    t_gradient_x, t_gradient_y = np.gradient(t_patch, axis=1), np.gradient(t_patch, axis=0)
    s_orientation, t_orientation = np.arctan2(s_gradient_y.mean(), s_gradient_x.mean()), np.arctan2(t_gradient_y.mean(), t_gradient_x.mean())
    s_patch_aligned = rotate(s_patch, -np.degrees(s_orientation), reshape=False, order=3)
    t_patch_aligned = rotate(t_patch, -np.degrees(t_orientation), reshape=False, order=3)
    return np.sqrt(np.sum((s_patch_aligned - t_patch_aligned) ** 2))

# As discussed, the algorithm seems to work for 90/180/270 rotations around the Z axis but does not generalize well (probably could come up with a better solution for calculating
#the gradients around arbitrary axes, and use reshaping to avoid cropping corners)

### Find the matches between two images

In [ ]:
def calculate_matches(source, target):
    # Getting the four best Harris-Stephens corners.
    # Feel free to use Shi-Tomasi or another number.
    s_points = harris_stephens_points(source, 4)
    t_points = harris_stephens_points(target, 4)

    # For each corner, analyzing a 3x3 patch around
    # it. Feel free to use another size if you want.
    size = 5

    # You are not expected to change the code below,
    # unless you want to get really creative.

    # Each match must be a pair of points, with the
    # first being the source point and the second
    # being the target point. Each point must be in
    # the (y, x) order, as we are used to in OpenCV.

    matches = []

    for s_y, s_x in s_points:
        s_patch = patch(source, s_y, s_x, size)

        min_dist = np.inf

        for t_y, t_x in t_points:
            t_patch = patch(target, t_y, t_x, size)

            dist = calculate_distance(s_patch, t_patch, size)

            if min_dist > dist:
                min_dist = dist
                min_pair = (t_y, t_x)

        matches.append(((s_y, s_x), min_pair))

    return matches

## Demonstration

### Loading the object that we want to detect

In [ ]:
source = cv_grayread('source.png', asfloat=True)

cv_imshow(source)

### Loading the scene where we want to detect the object

In [ ]:
baseline = cv_grayread('baseline.png', asfloat=True)

cv_imshow(baseline)

### Finding the matches

In [ ]:
matches = calculate_matches(source, baseline)

draw_matches(source, baseline, matches)

### Testing the framework with other images

In [ ]:
NAMES = [
    'brightness-low',
    'brightness-high',
    'contrast-low',
    'contrast-high',
    'noise',
    'rotation-90',
    'rotation-180',
    'rotation-270',
    'rotation-45',
    'rotation-135',
    'rotation-225',
    'rotation-315',
    'warp-top',
    'warp-bottom',
    'warp-left',
    'warp-right',
]

In [ ]:
for name in NAMES:
    target = cv_grayread(f'{name}.png', asfloat=True)
    matches = calculate_matches(source, target)
    draw_matches(source, target, matches)

You can click on the toc.png tab to the left to browse by section.